In [1]:
import os
import pandas as pd

import numpy as np

import sys
sys.path.append('../../')
from src.models._base import BaseModel
from src.utils import read_json_df
import random
from typing import Iterable, Union
import networkx as nx
import sklearn.metrics.pairwise
from scipy import sparse
import warnings

from src.models.median_baseline import MedianBaselineModel

from src.models.page_rank import PageRankModel
median = MedianBaselineModel()
page_rank = PageRankModel()


warnings.filterwarnings("ignore")

In [2]:
review_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_review.json")
user_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_user.json")
business_df = read_json_df("../../data/yelp_dataset/yelp_academic_dataset_business.json")

### Let's took 10k users and 10k bussiness from our dataset only

In [3]:

review_df = review_df[review_df.business_id.isin(random.sample(list(review_df.business_id.unique()),10000))]
review_df = review_df[review_df.user_id.isin(random.sample(list(review_df.user_id.to_list()),1000))]
review_df = review_df.groupby(['user_id','business_id']).agg({'stars':'last','date':'last'}).reset_index() # Drop Duplicates


In [4]:
review_df


,user_id,business_id,stars,date
0,--ZSfjyws1G8_8esn45CqA,YW9kZK0fpgb8xAn98V3pyg,1.0,2013-12-06 22:19:54
1,-0Q9kNv6aGzDeLqTJVQOrg,JozdEQAPQPzafLC8guMzfQ,1.0,2018-01-02 00:05:24
2,-0Q9kNv6aGzDeLqTJVQOrg,Ra9zH1CBYtAC6vMfKYsHew,4.0,2021-05-06 05:52:52
3,-2Pn4YRNRgJM8foLYkkv7g,Ksz_jsyyK3Zjlq_Cz81ZBA,5.0,2012-08-31 02:21:22
4,-4IniEqY0JOpucm7emnQxA,tcnXO16nGdIvM0EfaHXGPA,4.0,2013-01-20 07:01:32
...,...,...,...,...
3954,zeD7wF1nxJFwsqXRMiZSRw,IDtLPgUrqorrpqSLdfMhZQ,5.0,2019-07-12 17:36:48
3955,zjkNgwEDnJPTm2w4Y_-M1Q,PRMD-xnN2WnGvTE3pc-gng,1.0,2016-03-17 02:18:06
3956,zjuPb-M2u7UsNGFmvhI-Yg,ajq1w3VWgh5BzcnJO9uFig,3.0,2021-08-17 13:57:27
3957,zpYe74-itYGnGyz1z0T69Q,t8e4ZUR8p0CEWgMLIYtBwA,5.0,2020-10-03 21:36:28


In [5]:
user_df = user_df[user_df.user_id.isin(review_df.user_id.unique())]
business_df = business_df[business_df.business_id.isin(review_df.business_id.unique())]


In [6]:
median = MedianBaselineModel()
page_rank = PageRankModel()

In [7]:
median.evaluate(review_df,user_df,business_df)


Evaluation fold: 100%|████████████████████████████| 5/5 [00:00<00:00,  7.23it/s]


{'rmse': 1.386735155362442,
 'mae': 1.025493171471927,
 'accuracy': 0.3016691957511381,
 'f1': 0.16770790914958095,
 'precision': 0.24243918843335005,
 'recall': 0.21138215627076407,
 'AP@1': 0.003964587548254825,
 'AP@3': 0.003594233094732879,
 'AP@K': 0.00321758720077778,
 'MAP@K': 0.003311353789938141}

In [8]:
page_rank.evaluate(review_df,user_df,business_df)


Evaluation fold: 100%|████████████████████████████| 5/5 [03:43<00:00, 44.77s/it]


{'rmse': 3.085178745843936,
 'mae': 2.8106221547799697,
 'accuracy': 0.09074355083459787,
 'f1': 0.03318140671692239,
 'precision': 0.8181487101669196,
 'recall': 0.2,
 'AP@1': 0.004118349848900133,
 'AP@3': 0.002449837348526524,
 'AP@K': 0.0017478366139083794,
 'MAP@K': 0.0022976614139585596}